## REAL ESTATE-PRICE PREDICT  

In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv("DATA.csv")

In [3]:
housing.head()

In [4]:
housing.describe()

# MISSING ATTRIBUTES

In [5]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
imputer.fit(housing)
x=imputer.transform(housing)
housing_tr = pd.DataFrame(x,columns=housing.columns)
imputer.statistics_


# TRAIN-TEST SPLITTING

In [6]:
from sklearn.model_selection import train_test_split
train_,test_set = train_test_split(housing,test_size=0.2,random_state=42)

In [7]:
from sklearn.model_selection import StratifiedShuffleSplit 
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42) 
for train_index, test_index in split.split(housing, housing['CHAS']): 
    strat_train_set = housing.loc[train_index] 
    strat_test_set = housing.loc[test_index]

## CORRELATION

In [8]:
corr_matrix=housing.corr()
corr_matrix['MEDV'].sort_values(ascending=False)

In [9]:
from pandas.plotting import scatter_matrix
attributes=['MEDV','RM','ZN','LSTAT']
scatter_matrix(housing[attributes],figsize=(12,8))

In [10]:
housing_tr = strat_train_set.drop("MEDV",axis=1)
housing_labels = strat_train_set["MEDV"].copy()

## CREATING PIPELINE

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
my_pipeline = Pipeline([('imputer',SimpleImputer(strategy='median')),
                        ('std_scaler',StandardScaler()),
                       ])
housing_num_tr = my_pipeline.fit_transform(housing_tr)

# CHOOSING MODEL

## MODEL-1

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
model_1 = LinearRegression()
model_1.fit(housing_num_tr, housing_labels)
lin_predictions=model_1.predict(housing_num_tr)
lin_mse=mean_squared_error(housing_labels,lin_predictions)
lin_rmse=np.sqrt(lin_mse)
lin_rmse


# Model_2

In [13]:
from sklearn.tree import DecisionTreeRegressor
model_2 = DecisionTreeRegressor()
model_2.fit(housing_num_tr, housing_labels)
dtree_predictions=model_2.predict(housing_num_tr)
dtree_mse=mean_squared_error(housing_labels,dtree_predictions)
dtree_rmse=np.sqrt(dtree_mse)
dtree_rmse

## Model_3

In [14]:
from sklearn.ensemble import RandomForestRegressor
model_3 = RandomForestRegressor()
model_3.fit(housing_num_tr, housing_labels)
random_regressor_predictions=model_3.predict(housing_num_tr)
rforest_mse=mean_squared_error(housing_labels,random_regressor_predictions)
rforest_rmse=np.sqrt(rforest_mse)
rforest_rmse

# CROSS VALIDATION 

In [15]:
from sklearn.model_selection import cross_val_score

scores_1 = cross_val_score(model_1,housing_num_tr,housing_labels,scoring='neg_mean_squared_error',cv=10)
rmse_scores_1 = np.sqrt(-scores_1)
rmse_scores_1

In [16]:
scores_2 = cross_val_score(model_2,housing_num_tr,housing_labels,scoring='neg_mean_squared_error',cv=10)
rmse_scores_2 = np.sqrt(-scores_2)
rmse_scores_2

In [17]:
scores_3 = cross_val_score(model_3,housing_num_tr,housing_labels,scoring='neg_mean_squared_error',cv=10)
rmse_scores_3 = np.sqrt(-scores_3)
rmse_scores_3

In [18]:
def print_scores(string, scores):
    print(f'{string}\n')
    print("Scores are:", scores)
    print('Mean:', scores.mean())
    print('Standard Deviation:', scores.std(), '\n')

print_scores('Model-Linear Regression', rmse_scores_1)
print_scores('Model-Decision Tree Regressor', rmse_scores_2)
print_scores('Model-Random Forest Regressor', rmse_scores_3)

## Model Testing on Test Data-Choosing Random Forest Regressor as our Model



In [19]:
x_test = strat_test_set.drop('MEDV',axis=1)
y_test = strat_test_set['MEDV'].copy()
x_test_prepared = my_pipeline.transform(x_test)
final_predictions = model_3.predict(x_test_prepared)
final_mse = mean_squared_error(y_test,final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

print(final_predictions,list(y_test))

## Saving our Model through Joblib and using it 


In [20]:
from joblib import dump,load
dump(model_3,'dragon.joblib')
model = load('dragon.joblib')

features = np.array([x_test_prepared[0]])
model.predict(features)